In [1]:
import GetOldTweets3 as got
import pandas as pd 
pd.set_option('display.max_colwidth', None)


In [2]:
from tqdm import tqdm
tab_regions=['Marseille','Paris','Lyon','Toulouse','Bordeaux','Rennes','France']
tab_queries=['covid','coronavirus','epidemie','virus','pneumonie']


tweet_ids,tweet_usernames,tweet_datetime,tweet_text,tweet_geo,tweet_retweets,tweet_city,tweet_keyword=[],[],[],[]\
                                                                                                      ,[],[],[],[]
for ville in tqdm(tab_regions,total=len(tab_regions)):
    for keyword in tab_queries :
        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword)\
                                                   .setSince("2019-12-31")\
                                                   .setUntil("2020-03-10")\
                                                   .setNear(ville)\
                                                   .setLang('fr')
                                                   #.setWithin(200)
        tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        
        tweet_ids+=[tweet.id for tweet in tweets]
        tweet_usernames+=[tweet.username for tweet in tweets]
        tweet_datetime+=[tweet.date for tweet in tweets]
        tweet_text+=[tweet.text for tweet in tweets]
        tweet_geo+=[tweet.geo for tweet in tweets]
        tweet_retweets+=[tweet.retweets for tweet in tweets]
        tweet_city+=[ville]*len(tweets)
        tweet_keyword+=[keyword]*len(tweets)


        #all_tweets+=tweet
df_tweets=pd.DataFrame(data={'tweet_id':tweet_ids,'username':tweet_usernames,'datetime':tweet_datetime,
                            'text':tweet_text,'geoloc':tweet_geo,'retweets':tweet_retweets,'city':tweet_city,
                            'keyword':tweet_keyword})
df_tweets

  0%|          | 0/7 [00:00<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=covid%20near%3A%22Marseille%22%20within%3A15mi%20since%3A2019-12-31%20until%3A2020-03-10&src=typd
Traceback (most recent call last):
  File "/home/hamza/anaconda3/envs/py37/lib/python3.7/site-packages/GetOldTweets3/manager/TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "/home/hamza/anaconda3/envs/py37/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/home/hamza/anaconda3/envs/py37/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/home/hamza/anaconda3/envs/py37/lib/python3.7/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/home/hamza/anaconda3/envs/py37/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/home/hamza/anaconda3/envs/py37/lib/python

TypeError: object of type 'NoneType' has no len()

In [11]:
df_tweets.geoloc.value_counts()

    6268
Name: geoloc, dtype: int64

In [7]:
df_tweets.to_csv('projects/filiere/saved_data/df_GOT.csv',index=False)

## compare to covid data 

In [22]:
from datetime import datetime
import pickle
from clean import clean
df_tweets= pd.read_csv('/home/hamza/3A_projets/filiere/saved_data/df_GOT.csv')
df_tweets['text']=df_tweets['text'].apply(lambda x: clean(x))

df_covid_emotions= pickle.load(open('/home/hamza/3A_projets/filiere/saved_data/df_covid_emotions.pkl','rb'))

In [23]:
def get_time (x) :
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
def get_time_getold (x) :
    return datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S')
df_covid_emotions['date'] = df_covid_emotions['datetime'].apply(get_time)
df_tweets['date'] = df_tweets['datetime'].apply(get_time_getold)


In [25]:
tab_index=[]
for index, rows in df_tweets.iterrows():
    date_ = rows['text']
    for k,date_em in enumerate(list(df_covid_emotions.text.values )) :
        if date_em == date_ :
            print('sentence A :' ,rows['text'] )
            print('sentence B :' ,df_covid_emotions['text'][k])
            tab_index.append(index)
            continue

sentence A : Ma richesse est phénoménale  !  !  !  !  Épouse moi  !  !  !  !  !  Covid_19fr coronavirus coronavirusfr
sentence B : Ma richesse est phénoménale  !  !  !  !  Épouse moi  !  !  !  !  !  Covid_19fr coronavirus coronavirusfr
sentence A : Cher , pour lutter contre l ' épidémie de Coronavirus, une ostension exceptionnelle du chef de saint Martial à _limoges pourrait être une belle manière de rappeler le sens premier des ostensions limousines pour lutter et protéger la population contre les épidémies
sentence B : Cher , pour lutter contre l ' épidémie de Coronavirus, une ostension exceptionnelle du chef de saint Martial à _limoges pourrait être une belle manière de rappeler le sens premier des ostensions limousines pour lutter et protéger la population contre les épidémies
sentence A : Ma richesse est phénoménale  !  !  !  !  Épouse moi  !  !  !  !  !  Covid_19fr coronavirus coronavirusfr
sentence B : Ma richesse est phénoménale  !  !  !  !  Épouse moi  !  !  !  !  !  Covid_19f

In [28]:
df_tweets.drop(tab_index, inplace = True ) 


In [31]:
df_tweets.reset_index(drop=True , inplace=True)
df_tweets.to_csv('/home/hamza/3A_projets/filiere/saved_data/df_got_preprocessed.csv',index=False)

In [32]:
df_tweets

,tweet_id,username,datetime,text,geoloc,retweets,city,keyword,date
0,1237134417124089862,Samir17Samir,2020-03-09 21:53:24+00:00,"Covid 19 et pour étanchéité du masque FFP2 obligé de commencer à me raser . Mais j’y vais par étape . Bouc d’abord, moustache ce week-end pour tournoi volley thème séries et nous Magnum",NaN,1,Marseille,covid,2020-03-09 21:53:24
1,1237068889554595848,benichou_eliane,2020-03-09 17:33:01+00:00,Quand Marine se fait prophétesseCovid,NaN,0,Marseille,covid,2020-03-09 17:33:01
2,1237056609861451778,MartineVassal,2020-03-09 16:44:13+00:00,"En raison du contexte sanitaire actuel Covid_19 Coronavirus, je tiens à être au plus près des établissements de santé et sociaux, ici à la Maison d ' Enfant du Mas Joyeux à Marseille .",NaN,21,Marseille,covid,2020-03-09 16:44:13
3,1236973673506013185,benichou_eliane,2020-03-09 11:14:40+00:00,"Pas de bisous , c’est dangereux ! On dit salut maintenant covid",NaN,0,Marseille,covid,2020-03-09 11:14:40
4,1236596034748514310,benichou_eliane,2020-03-08 10:14:04+00:00,Le covid est il juif chretien ou musulman ou athée ? Chère madame,NaN,0,Marseille,covid,2020-03-08 10:14:04
...,...,...,...,...,...,...,...,...,...
6249,1221123525018300417,Claudyalnbll,2020-01-25 17:31:50+00:00,Votre histoire de virus là je vais rester chez moi barricadé pardon !,NaN,0,Rennes,virus,2020-01-25 17:31:50
6250,1220990496761417734,luc_jlh,2020-01-25 08:43:13+00:00,"Attention, la propagation de ce virus peut être très sournoise",NaN,0,Rennes,virus,2020-01-25 08:43:13
6251,1220986641617248258,ylefaou,2020-01-25 08:27:54+00:00,A lire attentivement pour limiter les mouvement de panic sur le nouveau virus . . . keep calm !,NaN,0,Rennes,virus,2020-01-25 08:27:54
6252,1220621494365147137,007_blaco,2020-01-24 08:16:56+00:00,Tout part en vrille ! La planète et maintenant c ' est des vilains virus ! !,NaN,0,Rennes,virus,2020-01-24 08:16:56
